# [LEGALST-190] Lab 2-15



This lab will provide an introduction to extra plugins in the folium package: Search and Draw.

*Estimated Time: 30 minutes*

---

### Table of Contents

[The Data](#section data)<br>

[Context](#section context)<br>

1 - [Search](#section 1)<br>

2 - [Draw](#section 2)<br>





**Dependencies:**

In [2]:
!pip install folium --upgrade
import folium
from folium import plugins
from folium.plugins import Draw
from folium.plugins import HeatMap

  Using cached branca-0.2.0-py3-none-any.whl
Requirement already up-to-date: six in /srv/app/venv/lib/python3.6/site-packages (from folium)
  Using cached requests-2.18.4-py2.py3-none-any.whl
Requirement already up-to-date: jinja2 in /srv/app/venv/lib/python3.6/site-packages (from folium)
  Using cached certifi-2018.1.18-py2.py3-none-any.whl
Requirement already up-to-date: urllib3<1.23,>=1.21.1 in /srv/app/venv/lib/python3.6/site-packages (from requests->folium)
Requirement already up-to-date: idna<2.7,>=2.5 in /srv/app/venv/lib/python3.6/site-packages (from requests->folium)
Requirement already up-to-date: chardet<3.1.0,>=3.0.2 in /srv/app/venv/lib/python3.6/site-packages (from requests->folium)
Requirement already up-to-date: MarkupSafe>=0.23 in /srv/app/venv/lib/python3.6/site-packages (from jinja2->folium)
  Found existing installation: certifi 2017.11.5
    Uninstalling certifi-2017.11.5:
      Successfully uninstalled certifi-2017.11.5
  Found existing installation: requests 2.12

In [3]:
from datascience import *
import json
import os
import pandas as pd
import numpy as np

## be extra sure that everything loads as expected

In [4]:
# %load search.py

from __future__ import (absolute_import, division, print_function)

from branca.element import CssLink, Figure, JavascriptLink, MacroElement

from jinja2 import Template


class Search(MacroElement):
    """
    Adds a search tool to your map.

    Parameters
    ----------
    data: str/JSON
        GeoJSON strings
    search_zoom: int
        zoom level when searching features, default 12
    search_label: str
        label to index the search, default 'name'
    geom_type: str
        geometry type, default 'Point'
    position: str
        Change the position of the search bar, can be:
        'topleft', 'topright', 'bottomright' or 'bottomleft',
        default 'topleft'

    See https://github.com/stefanocudini/leaflet-search for more information.

    """
    def __init__(self, data, search_zoom=12, search_label='name', geom_type='Point', position='topleft'):
        super(Search, self).__init__()
        self.position = position
        self.data = data
        self.search_label = search_label
        self.search_zoom = search_zoom
        self.geom_type = geom_type

        self._template = Template("""
        {% macro script(this, kwargs) %}

            var {{this.get_name()}} = new L.GeoJSON({{this.data}});

            {{this._parent.get_name()}}.addLayer({{this.get_name()}});

            var searchControl = new L.Control.Search({
                layer: {{this.get_name()}},
                propertyName: '{{this.search_label}}',
            {% if this.geom_type == 'Point' %}
                initial: false,
                zoom: {{this.search_zoom}},
                position:'{{this.position}}',
                hideMarkerOnCollapse: true
            {% endif %}
            {% if this.geom_type == 'Polygon' %}
                marker: false,
                moveToLocation: function(latlng, title, map) {
                var zoom = {{this._parent.get_name()}}.getBoundsZoom(latlng.layer.getBounds());
                    {{this._parent.get_name()}}.setView(latlng, zoom); // access the zoom
                }
            {% endif %}
                });
                searchControl.on('search:locationfound', function(e) {

                    e.layer.setStyle({fillColor: '#3f0', color: '#0f0'});
                    if(e.layer._popup)
                        e.layer.openPopup();

                }).on('search:collapsed', function(e) {

                    {{this.get_name()}}.eachLayer(function(layer) {   //restore feature color
                        {{this.get_name()}}.resetStyle(layer);
                    });
                });
            {{this._parent.get_name()}}.addControl( searchControl );

        {% endmacro %}
        """)  # noqa

    def render(self, **kwargs):
        super(Search, self).render()

        figure = self.get_root()
        assert isinstance(figure, Figure), ('You cannot render this Element '
                                            'if it is not in a Figure.')

        figure.header.add_child(
            JavascriptLink('https://cdn.jsdelivr.net/npm/leaflet-search@2.3.6/dist/leaflet-search.min.js'),  # noqa
            name='Leaflet.Search.js'
        )

        figure.header.add_child(
            CssLink('https://cdn.jsdelivr.net/npm/leaflet-search@2.3.6/dist/leaflet-search.min.css'),  # noqa
            name='Leaflet.Search.css'
        )


---

## The Data <a id='data'></a>
---

In [5]:
# Searchable US states
with open(os.path.join('data', 'search_states.json')) as f:
    states = json.loads(f.read())


def getColor(d):
    if d > 1000:
        return '#800026'
    if d > 500:
        return '#BD0026'
    if d > 200:
        return '#E31A1C'
    if d > 100:
        return '#FC4E2A'
    if d > 50:
        return '#FD8D3C'
    if d > 20:
        return '#FEB24C'
    if d > 10:
        return '#FED976'
    else:
        return '#FFEDA0'


for state in states['features']:
    state['properties']['color'] = getColor(state['properties']['density'])

# Searchable Roman location markers
with open(os.path.join('data', 'search_bars_rome.json')) as f:
    rome = json.loads(f.read())
    
# Berkeley PD call report location coordinates
with open('data/call_locs.txt') as f:
    coords = np.fromfile(f).tolist()
    call_locs = [[coords[i], coords[i + 1]] for i in range(0, len(coords), 2)]

---

## Plugins <a id='data'></a>
Now that we're familiar with several different types of Folium mapping, we make our maps more functional with two additional **plug-ins**: Folium package components that add new features. Today, we'll work with **Search** and **Draw**.


---


## Search  <a id='section 1'></a>

Most map applications you've used likely contained a search function. Here's how to add one using Folium.

Create a map of the United States. Next, create a Search object by calling its constructor function, then add the Search to your Map using `add_too`- just like you've done with many other Folium modules. 

`Search(...)` needs to be given some searchable GeoJSON-formatted data. Here, you can use `states`. 

Hint: Remember, the `add_too` function call looks something like `<thing you want to add>.add_to(<place you want to add it to>)`

In [6]:
us_coordinates = [39.8283, -98.5795]

usmap = folium.Map(us_coordinates, zoom_start=4)
Search(states, search_zoom=10, geom_type='Polygon').add_to(usmap)

usmap

You can also set optional parameters such as the location of the bar. We loaded the code for Search in an earlier cell: look at the 'parameters' section to see what options you can change, then see what happens when you change `geom_type`, `search_zoom`, and `position` on your states search map. 

In addition to searching for whole geographic areas, we can also search for specific locations. Create another searchable map in the cell below, this time using [41.9, 12.5] for the map coordinates and `rome` for your search data. You'll want to set a high `search_zoom`. How many cafes can you find?

In [7]:
# create a searchable map of Rome

rome_coordinates = [41.9028, 12.4964]
romemap = folium.Map(rome_coordinates, zoom_start=12)
Search(rome, search_zoom=10, geom_type='Point').add_to(romemap)
print(rome)
romemap

{'type': 'FeatureCollection', 'generator': 'overpass-turbo', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.', 'timestamp': '2015-08-08T19:03:02Z', 'features': [{'type': 'Feature', 'id': 'node/500129236', 'properties': {'@id': 'node/500129236', 'addr:city': 'Roma', 'addr:country': 'IT', 'addr:housenumber': '5', 'addr:postcode': '00184', 'addr:street': 'Piazza della Madonna dei Monti', 'amenity': 'bar', 'name': "la Bottega del caffe'"}, 'geometry': {'type': 'Point', 'coordinates': [12.4910927, 41.8950196]}}, {'type': 'Feature', 'id': 'node/560446888', 'properties': {'@id': 'node/560446888', 'amenity': 'bar', 'name': 'Marani'}, 'geometry': {'type': 'Point', 'coordinates': [12.5132952, 41.8970607]}}, {'type': 'Feature', 'id': 'node/574326909', 'properties': {'@id': 'node/574326909', 'amenity': 'bar', 'name': 'Il Faraone'}, 'geometry': {'type': 'Point', 'coordinates': [12.4910603, 41.8948257]}}, {'type': 'Feature', 'id'

Folium's Search is currently pretty limited, but it can be useful when you have the right set of GeoJSON data. 
1. Go to http://geojson.io, a tool for creating and downloading GeoJSON (the interface might look familiar: they use Leaflet, the same software Folium is build around).  
2. Create ten markers any where you'd like. Click on each marker and name them by adding a row with 'name' in the left column and the marker's name in the right column.
3. Save your markers in a GeoJSON file by using the save menu in the upper right.
4. Upload your GeoJSON file to datahub
5. Open your file in this notebook and create a searchable map using your markers. Hint: there's an example of code for opening a GeoJSON file earlier in this notebook.


In [8]:
# upload and plot your markers
# Searchable Berkeley in the 1990s location markers
with open(os.path.join('data', 'berklocs.geojson')) as f:
    berkeley90s = json.loads(f.read())
# create a searchable map of Berkeley

berkeley_coordinates = [37.8719, -122.2585]
berkmap = folium.Map(berkeley_coordinates, zoom_start=12)
Search(berkeley90s, search_zoom=14, geom_type='Point').add_to(berkmap)
print(berkeley90s)
berkmap

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Wall Berlin'}, 'geometry': {'type': 'Point', 'coordinates': [-122.25845575332642, 37.868044127072636]}}, {'type': 'Feature', 'properties': {'name': 'Kingpin Donuts'}, 'geometry': {'type': 'Point', 'coordinates': [-122.25794613361357, 37.868086475201736]}}, {'type': 'Feature', 'properties': {'name': 'GouBuLi'}, 'geometry': {'type': 'Point', 'coordinates': [-122.25980490446092, 37.86807588817176]}}, {'type': 'Feature', 'properties': {'name': 'Barrows Common Room'}, 'geometry': {'type': 'Point', 'coordinates': [-122.25749820470809, 37.87016150371354]}}, {'type': 'Feature', 'properties': {'name': "Nelson Polsby's Office"}, 'geometry': {'type': 'Point', 'coordinates': [-122.25817680358887, 37.87068872103537]}}, {'type': 'Feature', 'properties': {'name': 'Jupiter'}, 'geometry': {'type': 'Point', 'coordinates': [-122.26754307746889, 37.86967027775357]}}, {'type': 'Feature', 'properties': {'name': 'Good Sand

---

## Draw <a id='section 2'></a>

A **Draw** tool can be a great way to interactively add, change, or highlight features on your map. Adding it is  simple:

1. Create a Map 
2. Create a Draw object by calling the `Draw()` constructor function 
3. Add your Draw object to your map using `add_to`



Folium's Draw has a lot of features. Let's try a few.

- Draw a polygon representing the [Bermuda Triangle](https://en.wikipedia.org/wiki/Bermuda_Triangle)
- The Deepwater Horizon oil spill in 2010 occurred in the nearby Gulf of Mexico. Look up the exact coordinates, add a marker for the center, and draw two circles: one representing the size of the spill at its smallest, and one at its largest.

In [9]:
# create a map of Caribbean/Gulf of Mexico
bermuda_coords = [32.3078, -64.7505]
bermap = folium.Map(bermuda_coords, zoom_start=12)
# create a Draw object by calling the Draw() constructor function
Draw().add_to(bermap)
bermap


- The Proclaimers say [they would walk 500 miles and they would walk 500 more just to be the man who walked 1000 miles to fall down at your door](https://en.wikipedia.org/wiki/I%27m_Gonna_Be_(500_Miles). Create a map with a marker at the Proclaimers' birthplace in Edinburgh, Scotland. Then make a circle with a radius of 1000 miles (about 1600 kilometers) centered at Edinburgh to see if the Proclaimers can reach your door.
- Vanessa Carlton lives in Nashville, Tennessee and says that [you know she'd walk 1000 miles if she could just see you](https://en.wikipedia.org/wiki/A_Thousand_Miles). Use the edit layers button to move your circle to be centered at Nashville and see if you're in Vanessa's range. Note: the circle changes size slightly when you move it south. Why?



In [11]:
edinburgh_coords = [55.9533, -3.1883]
nashville_coords = [36.174465, -86.767960]
songmap = folium.Map(edinburgh_coords, zoom_start=2)
folium.Marker(edinburgh_coords, popup = "The Proclaimers").add_to(songmap)
folium.Marker(nashville_coords, popup = "Vanessa Carlton").add_to(songmap)
# create a Draw object by calling the Draw() constructor function
Draw().add_to(songmap)
songmap
#I don't get how to draw a circle of a given radius around a marker--that could be useful!

- Recreate the HeatMap you made on Tuesday using the Berkeley Police Department call location data (the call coordinates are stored in the variable `call_locs`). Add the Draw tool, then draw a polyline representing the route from Barrows Hall to your home that passes through the fewest call locations. How long is it? How does it compare to the shortest possible route?


In [15]:
#created a basemap of berkeley above called berkmap

#Create a Heatmap with the call data.
berk_heat_map = HeatMap(call_locs, radius=10)

# Add it to your Berkeley map.
berkmap.add_child(berk_heat_map)

#create a Draw() object
Draw().add_to(berkmap)

berkmap

Visualization is a powerful way to understand and communicate data. For each Draw tool, think of one data analysis project that would be a good use of the tool:

**ANSWER:**

Congratulations; you're a folium expert!

---
Notebook developed by: Keeley Takimoto

Data Science Modules: http://data.berkeley.edu/education/modules
